In [1]:
import requests
import pandas as pd
import time
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.10f' % x)

### Pulling Data From API

This code is commented out to prevent overwriting the original dataset with an empty dataset due to reaching the max API requests

In [2]:
# url = "https://zillow56.p.rapidapi.com/search"

# with open('api.txt', 'r') as file:
#     api_key = file.read()

# headers = {
# 	"X-RapidAPI-Key": api_key,
# 	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
# }

# lst_1 = []

# for page in range(1, 21):
#     querystring = {"location":"Utah County, UT", 
#                 "sortSelection":"days", 
#                 "page":f"{page}"}

#     response = requests.get(url, headers=headers, params=querystring)

#     data = response.json()
#     df__ = pd.json_normalize(data['results'])
#     lst_1.append(df__)
#     time.sleep(2)

# df = pd.concat(lst_1)

# df.to_csv('zillow_orig.csv')

### Scraping the current 30 year fixed mortgage rate

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
url = 'https://www.usbank.com/home-loans/mortgage/mortgage-rates/utah.html'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

element = driver.find_elements(By.XPATH, '/html/body/div[2]/div/div/div/div[4]/div/div[3]/section/div/div/div/div/div/div/section/div/div/div/div/div/div[1]/div[2]/span[1]')

In [7]:
todays_30_year_rate = element[0].text

with open('current_rate.txt', "w") as file:
    file.write(todays_30_year_rate)

### Cleaning the Data

In [8]:
df_orig = pd.read_csv('zillow_orig.csv')

with open('current_rate.txt', "r") as file:
    todays_30_year_rate = file.read()

In [10]:
# Dropping columns and rows of no interest
df = df_orig[['price', 'bathrooms', 'bedrooms', 'city', 'homeType', 'livingArea', 'zipcode', 'priceReduction', 'daysOnZillow', 'latitude', 'longitude']]
df = df[~df['homeType'].isin(['LOT', 'MANUFACTURED', 'MULTI_FAMILY'])]

In [11]:
# Calculating an estimate for a 30 year mortgage payment
rate = float(re.search(r'\d{1,2}\.\d+', todays_30_year_rate).group())/100

def calculate_30_year_payment(price):
    monthly_rate = rate/12
    M = price * (monthly_rate * (1 + monthly_rate)**360)/(((1 + monthly_rate)**360) - 1)
    return M

df['30_year_mortgage'] = df['price'].apply(calculate_30_year_payment)

In [12]:
# df.to_csv('zillow.csv')